<a href="https://colab.research.google.com/github/Kennenzz/predict-employee-attrition/blob/main/Train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


**XÂY DỰNG MÔ HÌNH DỰ ĐOÁN NGUY CƠ NGHỈ VIỆC CỦA NHÂN VIÊN**
---

**#Import các thư viện sẽ sử dụng trong đồ án bao gồm:**
*   **Pandas**
*   **Numpy**
*   **Matplotlib**
*   **Seaborn**

In [ ]:
import pandas as pd
import joblib
import math
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from pytorch_tabnet.tab_model import TabNetClassifier
import lightgbm as lgb
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier

ModuleNotFoundError: No module named 'pytorch_tabnet'

---
**Đọc file dataset**

In [ ]:
try:
    df = pd.read_csv("Dataset.csv")
    display(df.head())
except FileNotFoundError:
    print("Lỗi: Không tìm thấy file dataset trong dữ liệu folder")
except Exception as e:
    print(f"Có lỗi xảy ra: {e}")

---
**Mô tả tổng quan về dataset**

In [ ]:
display(df.info())
display(df.describe())
display(df.shape)

**Tổng quan: Dataset thông tin của nhân viên gồm có 1470 mẫu và 35 cột (thuộc tinh)**


---
# Phân loại các biến trong dataset

In [ ]:
# cần run file từ đầu đến cuối
numerical_cols = df.select_dtypes(include=np.number).columns
categorical_cols = df.select_dtypes(include='object').columns

print("Biến số:")
display(numerical_cols)

print("\nBiến phân loại:")
display(categorical_cols)

---
**Tiền xử lý dữ liệu**

In [ ]:
# Xử lý chuẩn hóa dữ liệu
for col in numerical_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')  # Chuyển đổi sang kiểu số, lỗi sẽ thành NaN
    if df[col].isnull().any():
        print(f"Cảnh báo: Cột '{col}' có giá trị không thể chuyển đổi sang số.")
    print(f"Cột '{col}' đã được chuyển đổi sang kiểu số.")


for col in categorical_cols:
    unique_values = df[col].unique()
    print(f"Giá trị duy nhất trong cột '{col}': {unique_values}")
    df[col] = df[col].str.strip().str.lower()   # Loại bỏ khoảng trắng và chuyển về chữ thường  
    unique_values_after = df[col].unique()
    print(f"Giá trị duy nhất trong cột '{col}' sau khi xử lý: {unique_values_after}")

Xử lý dữ liệu thiếu

In [ ]:
df.isnull().sum()


*Không có dữ liệu thiếu trong dataset*

*Xử lý dữ liệu dư thừa, dữ liệu gây nhiễu (không liên quan)*

In [ ]:
df.drop(columns=['Over18', 'EmployeeNumber', 'EmployeeCount','StandardHours'], axis =1, inplace= True)
# Cập nhật lại danh sách biến phân loại sau khi loại bỏ cột
numerical_cols = df.select_dtypes(include=np.number).columns
categorical_cols = df.select_dtypes(include='object').columns

Qua quan sát dữ liệu thì có thể thấy các cột dữ liệu không cần thiết như sau:
*   **Over18** tất cả nhân viên đều đã trên 18 tuổi
*   **EmployeeCount** luôn có giá trị là 1
*   **EmployeeNumber** luôn có giá trị khác nhau như việc nhân viên có mã nhân viên
*   **StandardHours** luôn có giá trị là 80

In [ ]:
display(df.info())
display(df.describe())
display(df.shape)

**Kiểu tra xem có dữ liệu trùng lặp hay không**

In [ ]:

duplicate_rows = df[df.duplicated()]
print(f"Số hàng trùng lặp: {duplicate_rows.shape[0]}")
print("Các hàng trùng lặp:")
display(duplicate_rows)


**Xử lý mã hóa biến phân loại**

In [ ]:
print("Các biến phân loại sau khi loại bỏ cột không cần thiết:")
display(categorical_cols)


*Mã hóa biến mục tiêu "Attrition" thành 0/1 và xử lý mã hóa các biến phân loại*

In [ ]:
label_encoder = LabelEncoder()
df['Attrition'] = label_encoder.fit_transform(df['Attrition'])

def mahoabienphanloai(encoder):
    for col in categorical_cols:
        if col not in ['Attrition','Over18']:  # Bỏ qua cột đã mã hóa hoặc không cần mã hóa
            df[col] = encoder.fit_transform(df[col])
mahoabienphanloai(label_encoder)

In [ ]:
display(df.info())
# hiện 100 dòng đầu tiên
display(df.head(100))

**Dùng IQR phát hiện các outliers và tạo ra bộ dữ liệu loại bỏ các outlier**

In [ ]:
# Phát hiện ngoại lệ sử dụng IQR
Q1 = df[numerical_cols].quantile(0.25)
Q3 = df[numerical_cols].quantile(0.75)
IQR = Q3 - Q1
print("IQR values:")
print(IQR)
outlier_condition = ((df[numerical_cols] < (Q1 - 1.5 * IQR)) | (df[numerical_cols] > (Q3 + 1.5 * IQR)))
outlier_indices = outlier_condition.any(axis=1)
print("Chỉ số của các hàng có ngoại lệ:")
print(outlier_indices[outlier_indices].index.tolist())
print(f"\nSố lượng ngoại lệ phát hiện: {outlier_indices.sum()}")
df_cleaned_outliers = df[~outlier_indices]
print(f"Số lượng mẫu ban đầu: {df.shape[0]}")
print(f"Số lượng mẫu sau khi loại bỏ ngoại lệ: {df_cleaned_outliers.shape[0]}")

**Xử lý mất cân bằng dữ liệu dùng phương pháp SMOTE**

In [ ]:
# Xử lý mất cân bằng dữ liệu sử dụng SMOTE (Chỉ áp dụng trên bộ dữ liệu gốc)
X = df.drop('Attrition', axis=1)
y = df['Attrition']
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)
print(f"Số lượng mẫu trước khi áp dụng SMOTE: {y.value_counts().to_dict()}")
print(f"Số lượng mẫu sau khi áp dụng SMOTE: {pd.Series(y_resampled).value_counts().to_dict()}")
df_resampled = pd.concat([X_resampled, y_resampled], axis=1)
display(df_resampled.info())

# dùng bộ dữ liệu sau khi loại bỏ ngoại lệ sử dụng SMOTE
X_cleaned_outliers = df_cleaned_outliers.drop('Attrition', axis=1)
y_cleaned_outliers = df_cleaned_outliers['Attrition']
smote_outliers = SMOTE(random_state=42)
X_resampled_cleaned_outliers, y_resampled_cleaned_outliers = smote_outliers.fit_resample(X_cleaned_outliers, y_cleaned_outliers)
print(f"Số lượng mẫu trước khi áp dụng SMOTE: {y_cleaned_outliers.value_counts().to_dict()}")
print(f"Số lượng mẫu sau khi áp dụng SMOTE: {pd.Series(y_resampled_cleaned_outliers).value_counts().to_dict()}")
df_resampled_outliers = pd.concat([X_resampled_cleaned_outliers, y_resampled_cleaned_outliers], axis=1)
display(df_resampled_outliers.info())

---
#**KHAI PHÁ DỮ LIỆU EDA**


*Tổng quan về tình hình nghỉ việc của nhân viên*

In [ ]:
# 1. Đếm số lượng nhân viên nghỉ việc và không nghỉ việc
attrition_counts = df['Attrition'].value_counts()
print("Số lượng nhân viên nghỉ việc và không nghỉ việc:")
display(attrition_counts)

# 2. Vẽ biểu đồ cột cho tỷ lệ nghỉ việc
plt.figure(figsize=(6, 4))
sns.countplot(data=df, x='Attrition', palette='viridis')
plt.title('Tỷ lệ nghỉ việc vs ở lại')
plt.xticks([0, 1], ['Ở lại', 'Nghỉ việc'])
plt.xlabel('Nghỉ việc')
plt.ylabel('Số lượng nhân viên')
plt.show()

# 3. Hiển thị phần trăm nhân viên nghỉ việc
if 1 in attrition_counts.index:
	attrition_percentage = (attrition_counts[1] / len(df)) * 100
	print(f"\nPhần trăm nhân viên nghỉ việc: {attrition_percentage:.2f}%")
else:
	print("\nKhông tìm thấy giá trị '1' trong cột 'Attrition'. Kiểm tra lại dữ liệu mã hóa.")

# làm lại với dữ liệu đã resample và clean outlier
attrition_counts_resampled = df_resampled_outliers['Attrition'].value_counts()
print("Số lượng nhân viên nghỉ việc và không nghỉ việc (sau khi resample và loại bỏ ngoại lệ):")
display(attrition_counts_resampled)
# vẽ biểu đồ
plt.figure(figsize=(6, 4))
sns.countplot(data=df_resampled_outliers, x='Attrition', palette='viridis')
plt.title('Tỷ lệ nghỉ việc vs ở lại (sau khi resample và loại bỏ ngoại lệ)')
plt.xticks([0, 1], ['Ở lại', 'Nghỉ việc'])
plt.xlabel('Nghỉ việc')
plt.ylabel('Số lượng nhân viên')
plt.show()

In [ ]:
# Lấy các biến số (loại trừ biến kiểu bool)
filtered_numerical_cols = df.select_dtypes(include=['int64', 'float64']).columns
# Tính số hàng và cột cho subplot
num_plots = len(filtered_numerical_cols)
cols = 5
rows = math.ceil(num_plots / cols)

plt.figure(figsize=(24, 18))
for idx, col in enumerate(filtered_numerical_cols):
    plt.subplot(rows, cols, idx + 1)
    sns.histplot(df[col], bins=20, kde=True, color='skyblue')
    plt.title(f'Biểu đồ Histogram: {col}')
plt.tight_layout()
plt.show()

# Vẽ countplot cho các biến phân loại (chỉ vẽ cho các biến đã được mã hóa nhị phân)
plt.figure(figsize=(24, 12))
for i, col in enumerate(df.columns[df.dtypes == 'bool']):
    plt.subplot(4, 5, i + 1)
    sns.countplot(x=col, data=df, palette='viridis')
    plt.title(f'Biểu đồ Countplot: {col}')
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(24, 18))
for idx, col in enumerate(filtered_numerical_cols):
    plt.subplot(rows, cols, idx + 1)
    sns.histplot(data=df, x=col, hue='Attrition', bins=20, kde=True, palette='viridis', alpha=0.7)
    plt.title(f'Histogram: {col} theo Attrition')
plt.tight_layout()
plt.show()

In [ ]:
# Vẽ countplot giữa các biến phân loại và biến mục tiêu Attrition
plt.figure(figsize=(24, 18))
for idx, col in enumerate(categorical_cols):
    plt.subplot(3, 3, idx + 1)
    # Nếu cột đã được one-hot encoding, vẽ countplot cho từng biến mới liên quan
    related_cols = [c for c in df.columns if c.startswith(col + "_") or c == col]
    for rel_col in related_cols:
        sns.countplot(data=df, x=rel_col, hue='Attrition', palette='viridis')
        plt.title(f'Countplot: {rel_col} vs Attrition')
        plt.xlabel(rel_col)
        plt.ylabel('Số lượng')
    plt.title(f'Countplot: {col} vs Attrition')
    plt.xlabel(col)
    plt.ylabel('Số lượng')
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(14, 6))

# Histogram for Age
plt.subplot(1, 2, 1)
sns.histplot(data=df, x='Age', hue='Attrition', bins=20, kde=True, palette='viridis', alpha=0.7)
plt.title('Phân phối Age theo Attrition')
plt.xlabel('Age')
plt.ylabel('Số lượng')

# Histogram for MonthlyIncome
plt.subplot(1, 2, 2)
sns.histplot(data=df, x='MonthlyIncome', hue='Attrition', bins=20, kde=True, palette='viridis', alpha=0.7)
plt.title('Phân phối MonthlyIncome theo Attrition')
plt.xlabel('MonthlyIncome')
plt.ylabel('Số lượng')

plt.tight_layout()
plt.show()

In [ ]:
# Tính ma trận tương quan
corr_matrix = df.corr()

# Vẽ heatmap với seaborn
plt.figure(figsize=(18, 14))
sns.heatmap(corr_matrix, annot=False, cmap='coolwarm', linewidths=0.5)
plt.title('Heatmap Ma trận tương quan các biến')
plt.show()

In [ ]:

print("Phân phối của biến mục tiêu 'Attrition':")
print(df['Attrition'].value_counts(normalize=True))
print(df['Attrition'].value_counts())

*Chia bộ dữ liệu ra làm 2 phần 70/30 để huấn luyện mô hình*

In [ ]:
# chia bộ dữ liệu ra làm 2 phần 70/30 để huấn luyện mô hình
X = df.drop('Attrition', axis=1)
y = df['Attrition']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)
print(f"Số lượng mẫu trong tập huấn luyện: {X_train.shape[0]}")
print(f"Số lượng mẫu trong tập kiểm tra: {X_test.shape[0]}")
print(f"Số lượng mẫu trong tập huấn luyện theo lớp:\n{y_train.value_counts()}")
print(f"Số lượng mẫu trong tập kiểm tra theo lớp:\n{y_test.value_counts()}")

# chia bộ dữ liệu đã qua resample và loại bỏ outliers ra làm 2 phần 70/30
X_resampled_cleaned_outliers_train, X_resampled_cleaned_outliers_test, y_resampled_cleaned_outliers_train, y_resampled_cleaned_outliers_test = train_test_split(
	X_resampled_cleaned_outliers, y_resampled_cleaned_outliers, test_size=0.3, random_state=42, stratify=y_resampled_cleaned_outliers
)
print(f"Số lượng mẫu trong tập huấn luyện (sau khi resample và loại bỏ ngoại lệ): {X_resampled_cleaned_outliers_train.shape[0]}")
print(f"Số lượng mẫu trong tập kiểm tra (sau khi resample và loại bỏ ngoại lệ): {X_resampled_cleaned_outliers_test.shape[0]}")
print(f"Số lượng mẫu trong tập huấn luyện theo lớp (sau khi resample và loại bỏ ngoại lệ):\n{y_resampled_cleaned_outliers_train.value_counts()}")
print(f"Số lượng mẫu trong tập kiểm tra theo lớp (sau khi resample và loại bỏ ngoại lệ):\n{y_resampled_cleaned_outliers_test.value_counts()}")

# chia bộ dữ liệu chỉ qua resample ra làm 2 phần 70/30
X_resampled_train, X_resampled_test, y_resampled_train, y_resampled_test = train_test_split(
	X_resampled, y_resampled, test_size=0.3, random_state=42, stratify=y_resampled
)
print(f"Số lượng mẫu trong tập huấn luyện (sau khi resample): {X_resampled_train.shape[0]}")
print(f"Số lượng mẫu trong tập kiểm tra (sau khi resample): {X_resampled_test.shape[0]}")
print(f"Số lượng mẫu trong tập huấn luyện theo lớp (sau khi resample):\n{y_resampled_train.value_counts()}")
print(f"Số lượng mẫu trong tập kiểm tra theo lớp (sau khi resample):\n{y_resampled_test.value_counts()}")


# Xây dựng mô hình (Build Model)

Xây dựng mô hình với bộ dữ liệu chưa qua resample bằng phương pháp SMOTE

In [ ]:
# Huấn luyện mô hình Logistic Regression và đánh giá
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print("Báo cáo phân loại:") 
print(classification_report(y_test, y_pred))
print("Ma trận nhầm lẫn:")
print(confusion_matrix(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))
print(f"Độ chính xác của mô hình: {accuracy_score(y_test, y_pred):.4f}")

In [ ]:
# Huấn luyện mô hình Decision Tree và đánh giá
model_dt = DecisionTreeClassifier(random_state=42)
model_dt.fit(X_train, y_train)

y_pred_dt = model_dt.predict(X_test)

print("Báo cáo phân loại:")
print(classification_report(y_test, y_pred_dt))
print("Ma trận nhầm lẫn:")
print(confusion_matrix(y_test, y_pred_dt))
print(f"Độ chính xác của mô hình: {accuracy_score(y_test, y_pred_dt):.4f}")


In [ ]:
#huấn luyện mô hình random forest
from sklearn.ensemble import RandomForestClassifier
model_rf = RandomForestClassifier(n_estimators=100, random_state=42)
model_rf.fit(X_train, y_train)
y_pred_rf = model_rf.predict(X_test)
print("Báo cáo phân loại:")
print(classification_report(y_test, y_pred_rf))
print("Ma trận nhầm lẫn:")
print(confusion_matrix(y_test, y_pred_rf))
print(f"Độ chính xác của mô hình: {accuracy_score(y_test, y_pred_rf):.4f}")

In [ ]:
#huấn luyện mô hình KNN (K-Nearest Neighbors) với điểm k chạy từ 1 tới 30
best_k = 1
best_accuracy = 0
for k in range(1, 31):
    model_knn = KNeighborsClassifier(n_neighbors=k)
    model_knn.fit(X_train, y_train)
    y_pred_knn = model_knn.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred_knn)
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_k = k
print(f"Giá trị k tốt nhất: {best_k} với độ chính xác: {best_accuracy:.4f}")
model_knn = KNeighborsClassifier(n_neighbors=best_k)
model_knn.fit(X_train, y_train)
y_pred_knn = model_knn.predict(X_test)
print("Báo cáo phân loại:")
print(classification_report(y_test, y_pred_knn))
print("Ma trận nhầm lẫn:")
print(confusion_matrix(y_test, y_pred_knn))
print(f"Độ chính xác của mô hình: {accuracy_score(y_test, y_pred_knn):.4f}")



In [ ]:
# Huấn luyện mô hình XGBoost với dữ liệu chỉ qua tiền xử lý 
model_xgb = XGBClassifier(eval_metric='logloss', use_label_encoder=False, random_state=42)
model_xgb.fit(X_train, y_train)
y_pred_xgb = model_xgb.predict(X_test)

print("Báo cáo phân loại:")
print(classification_report(y_test, y_pred_xgb))
print("Ma trận nhầm lẫn:")
print(confusion_matrix(y_test, y_pred_xgb))
print(f"Độ chính xác của mô hình: {accuracy_score(y_test, y_pred_xgb):.4f}")

In [ ]:
# Huấn luyện mô hình LightGBM và đánh giá
model_lgb = lgb.LGBMClassifier(random_state=42)
model_lgb.fit(X_train, y_train)
y_pred_lgb = model_lgb.predict(X_test)
print("Báo cáo phân loại:")
print(classification_report(y_test, y_pred_lgb))
print("Ma trận nhầm lẫn:")
print(confusion_matrix(y_test, y_pred_lgb))
print(f"Độ chính xác của mô hình: {accuracy_score(y_test, y_pred_lgb):.4f}")


In [ ]:
# Huấn luyện mô hình SVMmodel_svm = SVC(random_state=42)
model_svm = SVC(random_state=42)
model_svm.fit(X_train, y_train)
y_pred_svm = model_svm.predict(X_test)
print("Báo cáo phân loại:")
print(classification_report(y_test, y_pred_svm))
print("Ma trận nhầm lẫn:")
print(confusion_matrix(y_test, y_pred_svm))
print(f"Độ chính xác của mô hình: {accuracy_score(y_test, y_pred_svm):.4f}")


In [ ]:
# Huấn luyện mô hình Tabnet
model_tabnet = TabNetClassifier(random_state=42)
model_tabnet.fit(X_train.values, y_train.values, max_epochs=100, patience=10, batch_size=1024, virtual_batch_size=128)
y_pred_tabnet = model_tabnet.predict(X_test.values)
print("Báo cáo phân loại:")
print(classification_report(y_test, y_pred_tabnet))
print("Ma trận nhầm lẫn:")
print(confusion_matrix(y_test, y_pred_tabnet))
print(f"Độ chính xác của mô hình: {accuracy_score(y_test, y_pred_tabnet):.4f}")


Xây dựng mô hình với bộ dữ liệu đã qua resample bằng phương pháp SMOTE

In [ ]:
#Dùng data đã qua resample để train model Logistic Regression
model = LogisticRegression(max_iter=1000)
model.fit(X_resampled_train, y_resampled_train)
y_pred = model.predict(X_resampled_test)
print("Báo cáo phân loại:")
print(classification_report(y_resampled_test, y_pred))
print("Ma trận nhầm lẫn:")
print(confusion_matrix(y_resampled_test, y_pred))
print(confusion_matrix(y_resampled_test, y_pred))
print(f"Độ chính xác của mô hình: {accuracy_score(y_resampled_test, y_pred):.4f}")

In [ ]:
# Huấn luyện mô hình Decision Tree và đánh giá với dữ liệu đã qua resample
model_dt = DecisionTreeClassifier(random_state=42)
model_dt.fit(X_resampled_train, y_resampled_train)

y_pred_dt = model_dt.predict(X_resampled_test)

print("Báo cáo phân loại:")
print(classification_report(y_resampled_test, y_pred_dt))
print("Ma trận nhầm lẫn:")
print(confusion_matrix(y_resampled_test, y_pred_dt))
print(f"Độ chính xác của mô hình: {accuracy_score(y_resampled_test, y_pred_dt):.4f}")

In [ ]:
# Huấn luyện mô hình RandomForest và đánh giá với dữ liệu đã qua resample
model_rf = RandomForestClassifier(n_estimators=100, random_state=42)
model_rf.fit(X_resampled_train, y_resampled_train)
y_pred_rf = model_rf.predict(X_resampled_test)
print("Báo cáo phân loại:")
print(classification_report(y_resampled_test, y_pred_rf))
print("Ma trận nhầm lẫn:")
print(confusion_matrix(y_resampled_test, y_pred_rf))
print(f"Độ chính xác của mô hình: {accuracy_score(y_resampled_test, y_pred_rf):.4f}")


In [ ]:
# Huấn luyện mô hình KNN với dữ liệu đã qua resample cho K chạy từ 1 đến 30
best_k = 1
best_accuracy = 0
for k in range(1, 31):
    model_knn = KNeighborsClassifier(n_neighbors=k)
    model_knn.fit(X_resampled_train, y_resampled_train)
    y_pred_knn = model_knn.predict(X_resampled_test)
    accuracy = accuracy_score(y_resampled_test, y_pred_knn)
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_k = k
print(f"Giá trị k tốt nhất: {best_k} với độ chính xác: {best_accuracy:.4f}")
model_knn = KNeighborsClassifier(n_neighbors=best_k)
model_knn.fit(X_resampled_train, y_resampled_train)
y_pred_knn = model_knn.predict(X_resampled_test)
print("Báo cáo phân loại:")
print(classification_report(y_resampled_test, y_pred_knn))
print("Ma trận nhầm lẫn:")
print(confusion_matrix(y_resampled_test, y_pred_knn))
print(f"Độ chính xác của mô hình: {accuracy_score(y_resampled_test, y_pred_knn):.4f}")


In [ ]:
# Huấn luyện mô hình XGBoost với dữ liệu đã qua resample
model_xgb = XGBClassifier(eval_metric='logloss', use_label_encoder=False, random_state=42)
model_xgb.fit(X_resampled_train, y_resampled_train)
y_pred_xgb = model_xgb.predict(X_resampled_test)

print("Báo cáo phân loại:")
print(classification_report(y_resampled_test, y_pred_xgb))
print("Ma trận nhầm lẫn:")
print(confusion_matrix(y_resampled_test, y_pred_xgb))
print(f"Độ chính xác của mô hình: {accuracy_score(y_resampled_test, y_pred_xgb):.4f}")

In [ ]:
# Huấn luyện mô hình LightGBM với dữ liệu đã qua resample
model_lgb = lgb.LGBMClassifier(random_state=42)
model_lgb.fit(X_resampled_train, y_resampled_train)
y_pred_lgb = model_lgb.predict(X_resampled_test)
print("Báo cáo phân loại:")
print(classification_report(y_resampled_test, y_pred_lgb))
print("Ma trận nhầm lẫn:")
print(confusion_matrix(y_resampled_test, y_pred_lgb))
print(f"Độ chính xác của mô hình: {accuracy_score(y_resampled_test, y_pred_lgb):.4f}")

In [ ]:
# Huấn luyện mô hình SVM với dữ liệu đã qua resample
model_svm = SVC(random_state=42)
model_svm.fit(X_resampled_train, y_resampled_train)
y_pred_svm = model_svm.predict(X_resampled_test)
print("Báo cáo phân loại:")
print(classification_report(y_resampled_test, y_pred_svm))
print("Ma trận nhầm lẫn:")
print(confusion_matrix(y_resampled_test, y_pred_svm))
print(f"Độ chính xác của mô hình: {accuracy_score(y_resampled_test, y_pred_svm):.4f}")

In [ ]:
# Huấn luyện mô hình Tabnet với dữ liệu đã qua resample
model_tabnet = TabNetClassifier(random_state=42)
model_tabnet.fit(X_resampled_train.values, y_resampled_train.values, max_epochs=100, patience=10, batch_size=1024, virtual_batch_size=128)
y_pred_tabnet = model_tabnet.predict(X_resampled_test.values)
print("Báo cáo phân loại:")
print(classification_report(y_resampled_test, y_pred_tabnet))
print("Ma trận nhầm lẫn:")
print(confusion_matrix(y_resampled_test, y_pred_tabnet))
print(f"Độ chính xác của mô hình: {accuracy_score(y_resampled_test, y_pred_tabnet):.4f}")

Xây dựng mô hình với bộ dữ liệu đã qua resample bằng phương pháp SMOTE và loại bỏ outliers

In [ ]:
# Dùng data đã qua resample và loại bỏ outliers để train model
model = LogisticRegression(max_iter=1000)
model.fit(X_resampled_cleaned_outliers_train, y_resampled_cleaned_outliers_train)
y_pred = model.predict(X_resampled_cleaned_outliers_test)
print("Báo cáo phân loại:")
print(classification_report(y_resampled_cleaned_outliers_test, y_pred))
print("Ma trận nhầm lẫn:")
print(confusion_matrix(y_resampled_cleaned_outliers_test, y_pred))
print(confusion_matrix(y_resampled_cleaned_outliers_test, y_pred))
print(f"Độ chính xác của mô hình: {accuracy_score(y_resampled_cleaned_outliers_test, y_pred):.4f}")

In [ ]:
# Dùng data đã qua resample và clean outliers để train mô hình decision tree
model_dt = DecisionTreeClassifier(random_state=42)
model_dt.fit(X_resampled_cleaned_outliers_train, y_resampled_cleaned_outliers_train)

y_pred_dt = model_dt.predict(X_resampled_cleaned_outliers_test)

print("Báo cáo phân loại:")
print(classification_report(y_resampled_cleaned_outliers_test, y_pred_dt))
print("Ma trận nhầm lẫn:")
print(confusion_matrix(y_resampled_cleaned_outliers_test, y_pred_dt))
print(f"Độ chính xác của mô hình: {accuracy_score(y_resampled_cleaned_outliers_test, y_pred_dt):.4f}")

In [ ]:
# Huấn luyện mô hình RandomForest và đánh giá với dữ liệu đã qua resample và loại bỏ outliers
model_rf = RandomForestClassifier(n_estimators=100, random_state=42)
model_rf.fit(X_resampled_cleaned_outliers_train, y_resampled_cleaned_outliers_train)
y_pred_rf = model_rf.predict(X_resampled_cleaned_outliers_test) 
print("Báo cáo phân loại:")
print(classification_report(y_resampled_cleaned_outliers_test, y_pred_rf))
print("Ma trận nhầm lẫn:")
print(confusion_matrix(y_resampled_cleaned_outliers_test, y_pred_rf))
print(f"Độ chính xác của mô hình: {accuracy_score(y_resampled_cleaned_outliers_test, y_pred_rf):.4f}")


In [ ]:
# Huan luyện mô hình KNN với dữ liệu đã qua resample và loại bỏ outliers cho K chạy từ 1 đến 30
best_k = 1
best_accuracy = 0
for k in range(1, 31):
    model_knn = KNeighborsClassifier(n_neighbors=k)
    model_knn.fit(X_resampled_cleaned_outliers_train, y_resampled_cleaned_outliers_train)
    y_pred_knn = model_knn.predict(X_resampled_cleaned_outliers_test)
    accuracy = accuracy_score(y_resampled_cleaned_outliers_test, y_pred_knn)
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_k = k
print(f"Giá trị k tốt nhất: {best_k} với độ chính xác: {best_accuracy:.4f}")
model_knn = KNeighborsClassifier(n_neighbors=best_k)   
model_knn.fit(X_resampled_cleaned_outliers_train, y_resampled_cleaned_outliers_train)
y_pred_knn = model_knn.predict(X_resampled_cleaned_outliers_test)
print("Báo cáo phân loại:")
print(classification_report(y_resampled_cleaned_outliers_test, y_pred_knn))
print("Ma trận nhầm lẫn:")
print(confusion_matrix(y_resampled_cleaned_outliers_test, y_pred_knn))
print(f"Độ chính xác của mô hình: {accuracy_score(y_resampled_cleaned_outliers_test, y_pred_knn):.4f}")


In [ ]:
# Huấn luyện mô hình XGBoost với dữ liệu đã qua resample và loại bỏ outliers
model_xgb = XGBClassifier(eval_metric='logloss', use_label_encoder=False, random_state=42)
model_xgb.fit(X_resampled_cleaned_outliers_train, y_resampled_cleaned_outliers_train)
y_pred_xgb = model_xgb.predict(X_resampled_cleaned_outliers_test)

print("Báo cáo phân loại:")
print(classification_report(y_resampled_cleaned_outliers_test, y_pred_xgb))
print("Ma trận nhầm lẫn:")
print(confusion_matrix(y_resampled_cleaned_outliers_test, y_pred_xgb))
print(f"Độ chính xác của mô hình: {accuracy_score(y_resampled_cleaned_outliers_test, y_pred_xgb):.4f}")

In [ ]:
# Huấn luyện mô hình LightGBM với dữ liệu đã qua resample và loại bỏ outliers
model_lgb = lgb.LGBMClassifier(random_state=42)
model_lgb.fit(X_resampled_cleaned_outliers_train, y_resampled_cleaned_outliers_train)
y_pred_lgb = model_lgb.predict(X_resampled_cleaned_outliers_test)
print("Báo cáo phân loại:") 
print(classification_report(y_resampled_cleaned_outliers_test, y_pred_lgb))
print("Ma trận nhầm lẫn:")
print(confusion_matrix(y_resampled_cleaned_outliers_test, y_pred_lgb))
print(f"Độ chính xác của mô hình: {accuracy_score(y_resampled_cleaned_outliers_test, y_pred_lgb):.4f}")

In [ ]:
# Huấn luyện mô hình SVM với dữ liệu đã qua resample và loại bỏ outliers
model_svm = SVC(random_state=42)
model_svm.fit(X_resampled_cleaned_outliers_train, y_resampled_cleaned_outliers_train)
y_pred_svm = model_svm.predict(X_resampled_cleaned_outliers_test)
print("Báo cáo phân loại:")
print(classification_report(y_resampled_cleaned_outliers_test, y_pred_svm))
print("Ma trận nhầm lẫn:")
print(confusion_matrix(y_resampled_cleaned_outliers_test, y_pred_svm))
print(f"Độ chính xác của mô hình: {accuracy_score(y_resampled_cleaned_outliers_test, y_pred_svm):.4f}")

In [ ]:
# Huấn luyện mô hình Tabnet với dữ liệu đã qua resample và loại bỏ outliers
model_tabnet = TabNetClassifier(random_state=42)
model_tabnet.fit(X_resampled_cleaned_outliers_train.values, y_resampled_cleaned_outliers_train.values, max_epochs=100, patience=10, batch_size=1024, virtual_batch_size=128)
y_pred_tabnet = model_tabnet.predict(X_resampled_cleaned_outliers_test.values)
print("Báo cáo phân loại:")
print(classification_report(y_resampled_cleaned_outliers_test, y_pred_tabnet))
print("Ma trận nhầm lẫn:")
print(confusion_matrix(y_resampled_cleaned_outliers_test, y_pred_tabnet))
print(f"Độ chính xác của mô hình: {accuracy_score(y_resampled_cleaned_outliers_test, y_pred_tabnet):.4f}")

# Tinh chỉnh các mô hình


In [ ]:

# Tinh chỉnh mô hình sử dụng GridSearchCV cho Random Forest và Logistic Regression
# GridSearch cho Random Forest
param_grid_rf = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10]
}
grid_rf = GridSearchCV(RandomForestClassifier(random_state=42), param_grid_rf, cv=5, scoring='accuracy', n_jobs=-1)
grid_rf.fit(X_resampled_train, y_resampled_train)
print(f"Best parameters for Random Forest: {grid_rf.best_params_}")
print(f"Best accuracy for Random Forest: {grid_rf.best_score_:.4f}")

# GridSearch cho Logistic Regression
param_grid_lr = {
    'C': [0.01, 0.1, 1, 10, 100],
    'solver': ['lbfgs', 'liblinear']
}
grid_lr = GridSearchCV(LogisticRegression(max_iter=1000), param_grid_lr, cv=5, scoring='accuracy', n_jobs=-1)
grid_lr.fit(X_resampled_train, y_resampled_train)
print(f"Best parameters for Logistic Regression: {grid_lr.best_params_}")
print(f"Best accuracy for Logistic Regression: {grid_lr.best_score_:.4f}")

# Đánh giá lại trên tập kiểm tra
best_rf = grid_rf.best_estimator_
best_lr = grid_lr.best_estimator_

y_pred_rf_best = best_rf.predict(X_resampled_test)
y_pred_lr_best = best_lr.predict(X_resampled_test)

print("\nRandom Forest sau tinh chỉnh:")
print(classification_report(y_resampled_test, y_pred_rf_best))
print(f"Độ chính xác: {accuracy_score(y_resampled_test, y_pred_rf_best):.4f}")

print("\nLogistic Regression sau tinh chỉnh:")
print(classification_report(y_resampled_test, y_pred_lr_best))
print(f"Độ chính xác: {accuracy_score(y_resampled_test, y_pred_lr_best):.4f}")

Lưu lại các mô hình đã qua huấn luyện và các dataset qua các giai đoạn tiền xử lý khác nhau

In [ ]:
# Lưu mô hình đã huấn luyện sử dụng joblib
joblib.dump(best_rf, 'best_random_forest_model.pkl')
joblib.dump(best_lr, 'best_logistic_regression_model.pkl')
joblib.dump(model_knn, 'best_knn_model.pkl')
joblib.dump(model_dt, 'best_decision_tree_model.pkl')
joblib.dump(model_tabnet, 'best_tabnet_model.pkl')
jotlib.dump(model_xgb, 'best_xgboost_model.pkl')
jotlib.dump(model_lgb, 'best_lightgbm_model.pkl')
jotlib.dump(model_svm, 'best_svm_model.pkl')
# Lưu các bộ dữ liệu đã xử lý
df.to_csv('processed_dataset.csv', index=False)
df_resampled.to_csv('resampled_dataset.csv', index=False)
df_resampled_outliers.to_csv('resampled_cleaned_outliers_dataset.csv', index=False)
print("Mô hình và bộ dữ liệu đã được lưu thành công.")
# Load mô hình và dự đoán lại để kiểm tra
loaded_rf = joblib.load('best_random_forest_model.pkl')
loaded_lr = joblib.load('best_logistic_regression_model.pkl')
loaded_knn = joblib.load('best_knn_model.pkl')
loaded_dt = joblib.load('best_decision_tree_model.pkl')
